d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

# Connecting to JDBC

Apache Spark&trade; and Databricks&reg; allow you to connect to a number of data stores using JDBC.
## In this lesson you:
* Read data from a JDBC connection 
* Parallelize your read operation to leverage distributed computation

## Audience
* Primary Audience: Data Engineers
* Additional Audiences: Data Scientists and Data Pipeline Engineers

## Prerequisites
* Web browser: Please use a <a href="https://docs.azuredatabricks.net/user-guide/supported-browsers.html#supported-browsers" target="_blank">supported browser</a>.
* Concept (optional): <a href="https://academy.databricks.com/collections/frontpage/products/dataframes" target="_blank">DataFrames course from Databricks Academy</a>

<iframe  
src="//fast.wistia.net/embed/iframe/i07uvaoqgh?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/i07uvaoqgh?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

-sandbox
## Java Database Connectivity

Java Database Connectivity (JDBC) is an application programming interface (API) that defines database connections in Java environments.  Spark is written in Scala, which runs on the Java Virtual Machine (JVM).  This makes JDBC the preferred method for connecting to data whenever possible. Hadoop, Hive, and MySQL all run on Java and easily interface with Spark clusters.

Databases are advanced technologies that benefit from decades of research and development. To leverage the inherent efficiencies of database engines, Spark uses an optimization called predicate pushdown.  **Predicate pushdown uses the database itself to handle certain parts of a query (the predicates).**  In mathematics and functional programming, a predicate is anything that returns a Boolean.  In SQL terms, this often refers to the `WHERE` clause.  Since the database is filtering data before it arrives on the Spark cluster, there's less data transfer across the network and fewer records for Spark to process.  Spark's Catalyst Optimizer includes predicate pushdown communicated through the JDBC API, making JDBC an ideal data source for Spark workloads.

In the road map for ETL, this is the **Extract and Validate** step:

<img src="https://files.training.databricks.com/images/eLearning/ETL-Part-1/ETL-Process-1.png" style="border: 1px solid #aaa; border-radius: 10px 10px 10px 10px; box-shadow: 5px 5px 5px #aaa"/>

### Recalling the Design Pattern

Recall the design pattern for connecting to data from the previous lesson:  
<br>
1. Define the connection point.
2. Define connection parameters such as access credentials.
3. Add necessary options. 

After adhering to this, read data using `spark.read.options(<option key>, <option value>).<connection_type>(<endpoint>)`.  The JDBC connection uses this same formula with added complexity over what was covered in the lesson.

<iframe  
src="//fast.wistia.net/embed/iframe/2clbjyxese?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/2clbjyxese?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

Run the cell below to set up your environment.

In [8]:
%run "./Includes/Classroom-Setup"

-sandbox
Run the cell below to confirm you are using the right driver.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Each notebook has a default language that appears in upper corner of the screen next to the notebook name, and you can easily switch between languages in a notebook. To change languages, start your cell with `%python`, `%scala`, `%sql`, or `%r`.

In [10]:
%scala
// run this regardless of language type
Class.forName("org.postgresql.Driver")

res3: Class[_] = class org.postgresql.Driver

Define your database connection criteria. In this case, you need the hostname, port, and database name. 

Access the database `training` via port `5432` of a Postgres server sitting at the endpoint `server1.databricks.training`.

Combine the connection criteria into a URL.

In [12]:
jdbcHostname = "server1.databricks.training"
jdbcPort = 5432
jdbcDatabase = "training"

jdbcUrl = "jdbc:postgresql://{0}:{1}/{2}".format(jdbcHostname, jdbcPort, jdbcDatabase)

Create a connection properties object with the username and password for the database.

In [14]:
connectionProps = {
  "user": "readonly",
  "password": "readonly"
}

Read from the database by passing the URL, table name, and connection properties into `spark.read.jdbc()`.

In [16]:
accountDF = spark.read.jdbc(url=jdbcUrl, table="Account", properties=connectionProps)
display(accountDF)

userID,screenName,location,friendsCount,followersCount,description,insertID,ETLID
4853441060,PhaniaLeCroic,CDMX,592,446,"🌷 Geekrly 🌷 Juego LoL 🌷 CDMX 🌷 Anime, Ecología 🌷 Diseño Gráfico, Ilustración 🌷 Prensa en @LaComikeria ✨belinda.nava@lacomikeria.com✨",103079216713,3153
3858240741,Jill_Jixel,"Illinois, USA",289,174,"lover of Kingdom hearts,Pokemon,Minecraft and animals",103079216714,3153
410164962,WaIterBazar,null,113,69195,"Sólo se vive una vez, no la cagues. #WalterBazar",103079216715,3153
928455103828385798,malloandree,null,407,68,Dra Veterinaria,103079216716,3153
789543005480968192,gaysakuma,fluffy boy connoisseur,380,269,"rits/ryan | they/he | ♊️ | retweet heavy, enstars, Pokemon, mysmes, mha, p5, & tweets between me & my friends",103079216717,3153
302878354,kleider_caicedo,Ocaña- Colombia,371,337,null,103079216718,3153
741607796,nyukyult,𝒋𝒂𝒚𝒏𝒆𝒔 ₊ 𝒎 𝒔,302,1577,ᵎ ᵎ ➯ . . . ‹ᵇᵘᵇᵘ ˡᵒᵛᵉ ᵇᵒᵗ› ♡⃗. ˚ @official_gncd ᶻᶻᶻ,103079216719,3153
521922662,soygaby_,null,2443,2769,[24] 🐼 ×24/10/15× 💟13/09/17,103079216720,3153
924150951195398144,ModelsInternat1,United States,27,19,Any inquires dm me.,103079216721,3153
24738840,_wendyhamilton,"Washington, DC",726,579,"“She was free in her wildness. She was a wanderess, a drop of free water. She belonged to no man and to no city.”",103079216722,3153


## Exercise 1: Parallelizing JDBC Connections

The command above was executed as a serial read through a single connection to the database. This works well for small data sets; at scale, parallel reads are necessary for optimal performance.

See the [Managing Parallelism](https://docs.databricks.com/spark/latest/data-sources/sql-databases.html#managing-parallelism) section of the Databricks documentation.

-sandbox
### Step 1: Find the Range of Values in the Data

Parallel JDBC reads entail assigning a range of values for a given partition to read from. The first step of this divide-and-conquer approach is to find bounds of the data.

Calculate the range of values in the `insertID` column of `accountDF`. Save the minimum to `dfMin` and the maximum to `dfMax`.  **This should be the number itself rather than a DataFrame that contains the number.**  Use `.first()` to get a Scala or Python object.

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** See the `min()` and `max()` functions in Python `pyspark.sql.functions` or Scala `org.apache.spark.sql.functions`.

In [19]:
from pyspark.sql.functions import min, max

In [20]:
# TODO
dfMin = accountDF.select(min("insertID")).first()[0]
dfMax = accountDF.select(max("insertID")).first()[0]

In [21]:
# TEST - Run this cell to test your solution

dbTest("ET1-P-04-01-01", 0, dfMin)
dbTest("ET1-P-04-01-02", 214748365087, dfMax)

print("Tests passed!")

Tests passed!

-sandbox
### Step 2: Define the Connection Parameters.

<a href="https://docs.databricks.com/spark/latest/data-sources/sql-databases.html#manage-parallelism" target="_blank">Referencing the documentation,</a> define the connection parameters for this read.  Use 12 partitions.

Save the results to `accountDFParallel`.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Setting the column for your parallel read introduces unexpected behavior due to a bug in Spark. To make sure Spark uses the capitalization of your column, use `'"insertID"'` for your column. <a href="https://github.com/apache/spark/pull/20370#issuecomment-359958843" target="_blank"> Monitor the issue here.</a>

In [23]:
help(spark.read.jdbc)

Help on method jdbc in module pyspark.sql.readwriter:

jdbc(url, table, column=None, lowerBound=None, upperBound=None, numPartitions=None, predicates=None, properties=None) method of pyspark.sql.readwriter.DataFrameReader instance
 Construct a :class:`DataFrame` representing the database table named ``table``
 accessible via JDBC URL ``url`` and connection ``properties``.
 
 Partitions of the table will be retrieved in parallel if either ``column`` or
 ``predicates`` is specified. ``lowerBound`, ``upperBound`` and ``numPartitions``
 is needed when ``column`` is specified.
 
 If both ``column`` and ``predicates`` are specified, ``column`` will be used.
 
 .. note:: Don't create too many partitions in parallel on a large cluster;
 otherwise Spark might crash your external database systems.
 
 :param url: a JDBC URL of the form ``jdbc:subprotocol:subname``
 :param table: the name of the table
 :param column: the name of an integer column that will be used for partitioning;
 if this parameter is specified, then ``numPartitions``, ``lowerBound``
 (inclusive), and ``upperBound`` (exclusive) will form partition strides
 for generated WHERE clause expressions used to split the column
 ``column`` evenly
 :param lowerBound: the minimum value of ``column`` used to decide partition stride
 :param upperBound: the maximum value of ``column`` used to decide partition stride
 :param numPartitions: the number of partitions
 :param predicates: a list of expressions suitable for inclusion in WHERE clauses;
 each one defines one partition of the :class:`DataFrame`
 :param properties: a dictionary of JDBC database connection arguments. Normally at
 least properties "user" and "password" with their corresponding values.
 For example { 'user' : 'SYSTEM', 'password' : 'mypassword' }
 :return: a DataFrame
 
 .. versionadded:: 1.4

In [24]:
# TODO
accountDFParallel = (spark.read.jdbc(url=jdbcUrl,
    table="Account",
    column='"insertID"',
    lowerBound=dfMin,
    upperBound=dfMax,
    numPartitions=12,
    properties=connectionProps))

In [25]:
# TEST - Run this cell to test your solution
dbTest("ET1-P-04-02-01", 12, accountDFParallel.rdd.getNumPartitions())

print("Tests passed!")

Tests passed!

### Step 3: Compare the Serial and Parallel Reads

Compare the two reads with the `%timeit` function.

Display the number of partitions in each DataFrame by running the following:

In [28]:
print(accountDF.rdd.getNumPartitions())
print(accountDFParallel.rdd.getNumPartitions())

1
12

Invoke `%timeit` followed by calling a `.describe()`, which computes summary statistics, on both `accountDF` and `accountDFParallel`.

In [30]:
%timeit accountDF.describe()

In [31]:
%timeit accountDFParallel.describe()

What is the difference between serial and parallel reads?  Note that your results vary drastically depending on the cluster and number of partitions you use

## Review

**Question:** What is JDBC?  
**Answer:** JDBC stands for Java Database Connectivity, and is a Java API for connecting to databases such as MySQL, Hive, and other data stores.

**Question:** How does Spark read from a JDBC connection by default?  
**Answer:** With a serial read.  With additional specifications, Spark conducts a faster, parallel read.  Parallel reads take full advantage of Spark's distributed architecture.

**Question:** What is the general design pattern for connecting to your data?  
**Answer:** The general design patter is as follows:
0. Define the connection point
0. Define connection parameters such as access credentials
0. Add necessary options such as for headers or parallelization

## Next Steps

Start the next lesson, [Applying Schemas to JSON Data]($./05-Applying-Schemas-to-JSON-Data ).

## Additional Topics & Resources

**Q:** My tool can't connect via JDBC.  Can I connect via <a href="https://en.wikipedia.org/wiki/Open_Database_Connectivity" target="_blank">ODBC instead</a>?  
**A:** Yes.  The best practice is generally to use JDBC connections wherever possible since Spark runs on the JVM.  In cases where JDBC is either not supported or is less performant, use the Simba ODBC driver instead.  See <a href="https://docs.databricks.com/user-guide/clusters/jdbc-odbc.html" target="_blank">the Databricks documentation on connecting BI tools</a> for more details.


**Q:** How can I connect my Spark cluster to Azure Cosmos DB?  
**A:** Microsoft has developed an Azure Cosmos DB Spark Connector. Start with the <a href="https://docs.azuredatabricks.net/spark/latest/data-sources/azure/cosmosdb-connector.html#azure-cosmos-db" target="_blank">Databricks Azure Cosmos DB</a> documentation.

-sandbox
&copy; 2019 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>